In [3]:
import numpy as np
import theano
import string
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
import os
from os import path

In [4]:
import theano.tensor as T
from brown import get_sentences_with_word2idx_limit_vocab

In [42]:
def sigmoid(x):
    return 1/(1+np.exp(-1*x))
class SkipGram():
    def __init__(self,V,D,context_sz):
        #vocab sz
        self.V = V
        #embedding sz
        self.D = D
        self.context_sz  = context_sz
    def init_weight(self,Wi,Wo):
        return np.random.randn(Wi,Wo)
    def get_pnw(self,X):
        word_counts = {}
        total_count = sum([len(x) for x in X])
        for x in X:
            for word in x:
                if word not in word_counts:
                    word_counts[word] = 0
                word_counts[word] += 1
        pnw = np.zeros(self.V)
        for i in xrange(2,self.V):
            pnw[i] = (word_counts[words]/total_count)**0.75
        return pnw
    def get_negative_samples(self,context,num_negative_samples):
        saved = {}
        for word in context:
            saved[word] = pnw[word]
            pnw[word]=0
        negative_samples = np.random.choice(
                    a=np.arange(self.V),
                    size=num_negative_samples,
                    replace=False,
                    p=pnw/np.sum(self.pnw)
        )
        for key,val in saved.iteritems():
            pnw[key] = val
        return negative_samples
    def fit(self,X,alpha=1e-5,mu=0.99,epochs=10,num_negative_samples=5,reg=0.99):
        #number of sentences
        N = len(X)
        V = self.V
        D = self.D
        context_sz = self.context_sz
        self.get_pnw(X)
        
        w1 = self.init_weight(V,D)
        w2 = self.init_weight(D,V)
        
        dw1 = np.zeros(V,D)
        dw2 = np.zeros(D,V)
        
        cost_per_epoch = {}
        
        for i in xrange(epochs):
            shuffled_sentences = shuffle(np.arange(N))
            epoch_cost = 0
            for j in shuffled_sentences:
                x = X[j] # one sentence
                
                #if length of the sentence is too small
                if len(x)<(2*context_sz+1):
                    continue
                    
                #choose random word from the sentence
                jj = np.random.choice(len(x))
                
                start = max(0,jj-context_sz)
                end = min(len(x),jj+context_sz+1)
                
                context = np.concatenate(x[start:jj],x[jj:end])
                context = np.array(set(context))
                
                negative_samples = get_negative_samples(context,num_negative_samples)
                
                h = w1[jj]
                
                #postive error
                posA = h.dot(w2[:,context])
                pos_pY = sigmoid(posA) 
                
                #negative eror
                negA = h.dot(w2[:,negative_samples])
                neg_pY = sigmoid(-1*negA)
                                 
                #cost calculation
                cost = -np.log(pos_err).sum() - np.log(neg_err).sum()
                epoch_cost += 1.0*cost/(len(context) + num_negative_samples)
                #gradient calculation                  
                
                #w2 gradient calculation
                pos_err = pos_pY - 1 
                w2[:,context] += mu*dw1[:,context] - alpha(np.outer(pos_err,h) + reg*w1[:,context])
                dw2[:,context] = np.outer(pos_err,h) + reg*w1[:,context]
                
                neg_err = 1 - neg_pY
                w2[:,negative_samples]  += mu*dw2[:,negative_samples] - alpha(np.outer(neg_err,h) + reg*w2[:,negative_samples])
                dw2[:,negative_samples] = np.outer(neg_err,h) + reg*w2[:,negative_samples]
                
                #w1 gradient calculation
                grad_w1_jj = np.dot(pos_err,np.transpose(w2[:,context])) + np.dot(neg_err,np.transpose(w2[:,negative_samples]))
                w1[ x[jj],:] += mu*dw1[x[jj],:] - alpha*(grad_w1_jj + reg*dw1[x[jj],:])
                dw1[jj,:] = grad_w1_jj + reg*dw1[jj,:]
                                                               
                
                
                                  

In [43]:
#def main():
sentences,word2idx = get_sentences_with_word2idx_limit_vocab(n_vocab=1000)
print len(sentences),len(word2idx)

('START', inf)
('END', inf)
(u'man', inf)
(u'paris', inf)
(u'britain', inf)
(u'england', inf)
(u'king', inf)
(u'woman', inf)
(u'rome', inf)
(u'london', inf)
(u'queen', inf)
(u'italy', inf)
(u'france', inf)
(u'the', 69971)
(u',', 58334)
(u'.', 49346)
(u'of', 36412)
(u'and', 28853)
(u'to', 26158)
(u'a', 23195)
(u'in', 21337)
(u'that', 10594)
(u'is', 10109)
(u'was', 9815)
(u'he', 9548)
(u'for', 9489)
(u'``', 8837)
(u"''", 8789)
(u'it', 8760)
(u'with', 7289)
(u'as', 7253)
(u'his', 6996)
(u'on', 6741)
(u'be', 6377)
(u';', 5566)
(u'at', 5372)
(u'by', 5306)
(u'i', 5164)
(u'this', 5145)
(u'had', 5133)
(u'?', 4693)
(u'not', 4610)
(u'are', 4394)
(u'but', 4381)
(u'from', 4370)
(u'or', 4206)
(u'have', 3942)
(u'an', 3740)
(u'they', 3620)
(u'which', 3561)
(u'--', 3432)
(u'one', 3292)
(u'you', 3286)
(u'were', 3284)
(u'her', 3036)
(u'all', 3001)
(u'she', 2860)
(u'there', 2728)
(u'would', 2714)
(u'their', 2669)
(u'we', 2652)
(u'him', 2619)
(u'been', 2472)
(u')', 2466)
(u'has', 2437)
(u'(', 2435)
(u'whe

In [25]:
lists = np.array([12,12,2,1])

In [34]:
np.array(set(lists))

array(set([1, 2, 12]), dtype=object)

TypeError: 'list' object is not callable